# Lab: Building a Multi-Agent Sales Team with Tools and Handoffs

In this advanced lab, we will use the OpenAI Agents SDK to construct a sophisticated, multi-agent system that automates the process of generating and sending cold sales outreach emails. 

This project will demonstrate several key agentic design patterns:
1.  **Agent Specialization**: Creating multiple agents, each with a unique persona and task.
2.  **Parallel Execution**: Running multiple agents simultaneously to generate diverse outputs.
3.  **Evaluator Agent**: Using one agent to review and select the best output from others.
4.  **Tools as Functions**: Converting a regular Python function into a tool that an agent can use.
5.  **Agents as Tools**: Converting an entire agent into a tool for another agent to use.
6.  **Handoffs**: Delegating a task from a manager agent to a subordinate agent to complete a workflow.

### Setup: SendGrid API for Sending Emails

This lab uses the SendGrid service to send emails programmatically. You will need a free account and an API key.

1.  **Create an Account**: Go to [SendGrid](https://sendgrid.com/) and sign up for a free account.
2.  **Create an API Key**: In the SendGrid dashboard, navigate to `Settings` > `API Keys` and click `Create API Key`. Give it a name and create the key.
3.  **Update `.env` file**: Copy the generated API key and add it to your `.env` file:
    ```
    SENDGRID_API_KEY=YOUR_API_KEY_HERE
    ```
4.  **Verify a Sender**: In SendGrid, go to `Settings` > `Sender Authentication`. You must verify a "Single Sender" (your own email address) from which SendGrid is allowed to send emails.

In [ ]:
# === Imports ===
import os
import asyncio
from typing import Dict
from dotenv import load_dotenv

# OpenAI Agents SDK components
from agents import Agent, Runner, trace, function_tool

# SendGrid for sending emails
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content

In [ ]:
load_dotenv(override=True)

In [ ]:
# === Test SendGrid Configuration ===
# Before proceeding, let's ensure your SendGrid setup is working correctly.

def send_test_email():
    # IMPORTANT: Replace these with your verified sender and recipient email addresses.
    FROM_EMAIL = "your-verified-sender@example.com" 
    TO_EMAIL = "your-recipient@example.com"
    
    try:
        sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
        from_email = Email(FROM_EMAIL)
        to_email = To(TO_EMAIL)
        subject = "SendGrid Test Email for Agentic AI Course"
        content = Content("text/plain", "This is a test email to confirm your SendGrid setup is working.")
        mail = Mail(from_email, to_email, subject, content)
        response = sg.client.mail.send.post(request_body=mail.get())
        
        if response.status_code == 202:
            print(f"Test email sent successfully (Status Code: {response.status_code}). Please check your inbox!")
        else:
            print(f"Error sending email. Status Code: {response.status_code}. Body: {response.body}")
            
    except Exception as e:
        print(f"An exception occurred: {e}")
        print("Please check your SENDGRID_API_KEY in the .env file and your verified sender email in SendGrid.")

# send_test_email() # Uncomment this line to run the test

### Part 1: Agent Specialization and Parallel Execution

We'll start by creating three distinct "Sales Agents," each with a different personality for writing cold outreach emails. We'll then run them in parallel to generate three email drafts simultaneously.

In [ ]:
# === Define Agent Personas ===
company_description = "a company that provides a SaaS tool called 'ComplAI' for ensuring SOC2 compliance and preparing for audits, powered by AI."

instructions1 = f"You are a highly professional sales agent for {company_description} You write formal, serious, and benefit-driven cold emails."
instructions2 = f"You are a witty and engaging sales agent for {company_description} You write humorous, personable cold emails that are likely to get a response."
instructions3 = f"You are a busy, no-nonsense sales agent for {company_description} You write concise, to-the-point cold emails that respect the reader's time."

In [ ]:
# === Create a Team of Specialized Agents ===
professional_agent = Agent(name="Professional_Sales_Agent", instructions=instructions1, model="gpt-4o-mini")
engaging_agent = Agent(name="Engaging_Sales_Agent", instructions=instructions2, model="gpt-4o-mini")
concise_agent = Agent(name="Busy_Sales_Agent", instructions=instructions3, model="gpt-4o-mini")

In [ ]:
# === Run Agents in Parallel and Select the Best Output ===

# Define an 'Evaluator' agent to pick the best email.
picker_agent = Agent(
    name="Email_Picker",
    instructions="You are a marketing manager. You will be given three draft emails. Your task is to select the single best one that is most likely to get a positive response from a busy executive. Do not explain your choice; reply only with the full text of the selected email.",
    model="gpt-4o-mini"
)

async def generate_and_select_email(prompt: str):
    with trace("Generate_and_Select_Email_Workflow"):
        # 1. Run all three sales agents in parallel.
        print("--- Generating email drafts in parallel... ---")
        results = await asyncio.gather(
            Runner.run(professional_agent, prompt),
            Runner.run(engaging_agent, prompt),
            Runner.run(concise_agent, prompt),
        )
        drafts = [result.final_output for result in results]

        # 2. Prepare the drafts for the picker agent.
        picker_prompt = "Here are three email drafts. Please select the best one:\n\n---\n\nDRAFT 1:\n" + drafts[0] + "\n\n---\n\nDRAFT 2:\n" + drafts[1] + "\n\n---\n\nDRAFT 3:\n" + drafts[2]

        # 3. Run the picker agent to select the best email.
        print("--- Selecting the best draft... ---")
        best_email_result = await Runner.run(picker_agent, picker_prompt)
        
        print("\n--- BEST EMAIL SELECTED ---")
        print(best_email_result.final_output)

# await generate_and_select_email("Write a cold sales email to the CEO of a tech startup.")

### Part 2: Creating and Using Tools

Now we'll introduce tools. We will create a tool from a Python function (`send_email`) and also convert our existing agents into tools that a manager agent can call upon.

In [ ]:
# === Create a Tool from a Python Function ===
# The `@function_tool` decorator automatically converts this Python function
# into a JSON schema that an LLM can understand and decide to call.

@function_tool
def send_email(body: str):
    """Sends an email with the given body to a predefined sales prospect."""
    print(f"--- Sending Email Tool Called... ---")
    # IMPORTANT: Replace these with your verified sender and recipient email addresses.
    FROM_EMAIL = "your-verified-sender@example.com"
    TO_EMAIL = "your-recipient@example.com"
    
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(FROM_EMAIL)
    to_email = To(TO_EMAIL)
    subject = "A message from ComplAI"
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, subject, content)
    sg.client.mail.send.post(request_body=mail.get())
    print("--- Email Sent Successfully ---")
    return {"status": "success"}

In [ ]:
# === Create Tools from Other Agents ===
# The `.as_tool()` method allows one agent to become a callable tool for another agent.
# This is how we build hierarchies and teams of agents.

description = "Writes a cold sales email draft from a specific personality. Use this to generate options."

tool1 = professional_agent.as_tool(tool_name="professional_email_writer", tool_description=description)
tool2 = engaging_agent.as_tool(tool_name="engaging_email_writer", tool_description=description)
tool3 = concise_agent.as_tool(tool_name="concise_email_writer", tool_description=description)

# This is the list of all tools available to our manager agent.
manager_tools = [tool1, tool2, tool3, send_email]

### Part 3: Orchestration with a Manager Agent

Now we create a "Sales Manager" agent. Its job is not to write emails itself, but to orchestrate the other agents and tools to complete a task.

In [ ]:
# === Define the Manager Agent ===
manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to orchestrate the sending of the single best cold sales email.

Follow these steps carefully:
1.  **Generate Drafts**: Use all three email writer tools (`professional_email_writer`, `engaging_email_writer`, `concise_email_writer`) to generate three different email drafts. Do not proceed until you have all three drafts.
2.  **Evaluate and Select**: Review the three drafts and choose the single best one based on your expert judgment of what is most effective.
3.  **Send the Email**: Use the `send_email` tool to send ONLY the winning email draft.

Crucial Rules:
- You must use the writer tools to generate drafts; do not write them yourself.
- You must send exactly ONE email using the `send_email` tool.
"""

sales_manager = Agent(
    name="Sales_Manager", 
    instructions=manager_instructions, 
    tools=manager_tools, 
    model="gpt-4o"
)

async def run_manager_workflow(prompt: str):
    with trace("Sales_Manager_Orchestration"):
        await Runner.run(sales_manager, prompt)

# await run_manager_workflow("Send a cold sales email addressed to 'Dear CEO' from Alice at ComplAI.")

### Part 4: Delegation with Handoffs

Handoffs are similar to tools but represent a more formal delegation of a task. Instead of calling a tool and getting a result back, the manager agent will hand off the final email body to a specialized "Emailer Agent" whose only job is to format and send it.

In [ ]:
# === Define a Specialized Emailer Agent ===

# Create tools for the emailer agent to use.
subject_writer = Agent(name="Subject_Writer", instructions="You write compelling, concise subjects for cold sales emails.", model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Writes a subject for an email based on its body.")

html_converter = Agent(name="HTML_Converter", instructions="You convert a plain text email body into a clean, professional HTML format.", model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Converts a plain text email body to HTML.")

@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """Sends an email with a subject and HTML body."""
    print("--- Sending HTML Email Tool Called... ---")
    FROM_EMAIL = "your-verified-sender@example.com"
    TO_EMAIL = "your-recipient@example.com"
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    mail = Mail(Email(FROM_EMAIL), To(TO_EMAIL), subject, Content("text/html", html_body))
    sg.client.mail.send.post(request_body=mail.get())
    print("--- HTML Email Sent Successfully ---")
    return {"status": "success"}

# Define the Emailer Agent that uses these tools.
emailer_instructions = """You are an email formatting and sending specialist. You receive the body of an email. 
1. First, use the `subject_writer` tool to create a subject.
2. Second, use the `html_converter` tool to format the body as HTML.
3. Finally, use the `send_html_email` tool to send the email.
"""
emailer_agent = Agent(
    name="Emailer_Agent",
    instructions=emailer_instructions,
    tools=[subject_tool, html_tool, send_html_email],
    model="gpt-4o",
    handoff_description="Takes a plain text email body, formats it, and sends it." # This description is key for the handoff.
)

In [ ]:
# === Define the Final Manager Agent with Handoffs ===

# The manager's tools are now just the email writers.
final_manager_tools = [tool1, tool2, tool3]
# The manager's handoff is the entire emailer agent.
final_manager_handoffs = [emailer_agent]

final_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to orchestrate the sending of the single best cold sales email.

Follow these steps carefully:
1.  **Generate Drafts**: Use all three email writer tools to generate three different email drafts.
2.  **Evaluate and Select**: Review the drafts and choose the single best one.
3.  **Handoff for Sending**: Pass ONLY the winning email draft to the 'Emailer_Agent'. The Emailer_Agent will handle all formatting and sending.

Crucial Rules:
- You must use the writer tools to generate drafts; do not write them yourself.
- You must hand off exactly ONE email to the Emailer_Agent.
"""

final_sales_manager = Agent(
    name="Final_Sales_Manager",
    instructions=final_manager_instructions,
    tools=final_manager_tools,
    handoffs=final_manager_handoffs,
    model="gpt-4o")

async def run_final_workflow(prompt: str):
    with trace("Full_Sales_Automation_with_Handoff"):
        await Runner.run(final_sales_manager, prompt)

await run_final_workflow("Send a cold sales email to the CEO of a new startup from Alex at ComplAI.")

### Final Check

Remember to check two places:

1.  **The Trace**: [https://platform.openai.com/traces](https://platform.openai.com/traces) to see the full, step-by-step execution of the agent team.
2.  **Your Email Inbox**: To see the final, formatted email!